<h2 align="center">Data Engineer Evaluation</h2>

## Deliverables

<div class="alert alert-block alert-info">
<div class="alert alert-block alert-danger">1. You have received a repo containing a folder named <b>Tests</b>. Please keep the structure of this folder as it is and do not modify anything in assets folder. You can add your own folders inside Tests folder.</div>
<div class="alert alert-block alert-warning">2. You have to submit dumps of both mongodb collections for idealista and metromadrid. To do so create a folder named <b>dbdumps</b> inside Tests folder and put you db dumps in that folder.</div>
<div class="alert alert-block alert-success">3. Once done with the code, commit your final changes and inform us by email.</div>
</div>

***

## Problem Statement

You are going to scrape following websites
1. idealista - https://www.idealista.com/maps/madrid-madrid/
2. metromadrid - https://ctmulti.metromadrid.es/ctmulti/accesytevisor/index.html

<div class="alert alert-block alert-info">
Both sites are offered in <b>multiple languages</b>. You can choose the language you are comfortable with.
</div>


## Approach(s)

## 1. idealista
***
Navigate to above mentioned idealista link you will find something like this
<img src="../Tests/assets/test_i_1.png" alt="Home" text="Enjoy your Test ;)" />
<h3 align="center">Picture-1</h3>

You can see a list of streets in the left pane. If you click any of the street, you will be redirected to the following page
<img src="../Tests/assets/test_i_2.png" alt="Buildings in Street" text="Hola"/>
<h3 align="center">Picture-2</h3>

As you can see in Picture-2 **0, 2, 4, 5, 7** are buildings in this particular streets.

When you click on any of these buildings you will find something like this
<img src="../Tests/assets/test_i_3.png" alt="picture 3" text="Print smiley face at the end of each task for 10 bonus points"/>
<h3 align="center">Picture-3</h3>

#### Your Goal
***
1. get first 150 streets
2. In each street, get all buildings
3. with in each building you will be scraping **chracteristics of the building** part in each building and saving it in mongodb collection.

<div class="alert alert-block alert-warning">
where in Picture-3 <b>id</b> is the first 14 digits of properties i.e. 0447208VK4704G and it will be unique against all properties. 
</div>

4. Once you have all the buildings in 150 streets in a database you have to create indexes for more efficiency 
5. Query the database to find and display following results:
    1. building id with maximum number of floors
    2. Number of buildings containing lifts
    3. Number of buildings containing storage
    4. Number of buildings containing both lift and storage
    5. Average floors in each building

***
***

## 2. metromadrid
***
Navigate to above mentioned metromadrid link you will find something like this
<img src="../Tests/assets/test_m_1.png" alt="Home" />
<h3 align="center">Picture-4</h3>

Here you will add source station name and destination station name and press result

You will get the following results
<img src="../Tests/assets/test_m_2.png" alt="Buildings in Street" text="Picture 2"/>
<h3 align="center">Picture-5</h3>

#### Your Goal
***
1. calculate the travel time from following list of stations to every other station in madrid and store it in mongo database

<img src="../Tests/assets/test_m_3_1.png" alt="Name of stations" text="use this link for easy scraping: https://ctmulti.metromadrid.es/ctmulti/rutasAA/index.html" align="left"/>

<h3 align="center">Picture-6</h3>

<div class="alert alert-block alert-warning">
<div>In Picture-6 the digits represent metro lines and against each line their are 2 stations. First station is the starting station and second one is the ending station</div>
<div>You need to construct your list of 10 stations from starting station of each line ignoring the 6th line since 6th line is a circular line and have no start and end point</div>
<div>For example ['Pinar de Chamartín', 'Las Rosas', 'Villaverde Alto', ...]</div>
</div>

<div class="alert alert-block alert-success">
You can get the list of all stations from the following link <a href="https://www.metromadrid.es/en/linea/linea-1">Metro Stations List</a>
</div>

<div class="alert alert-block alert-warning">
Most of the stations will be repeating in different lines so you have to get all the stations in each metro line and create a list of unique stations
</div>

2. Answer the following Questions
    1. Total number of metro stations in Madrid?
    2. Which station is nearest to **Pinar de Chamartín** station in terms of time?
    3. Which station is farthest to **Pinar de Chamartín** station in terms of time?
    4. What is the average time from **Las Rosas** station to any other station?

<div class="alert alert-block alert-danger">
<b>Alert:</b> Do not start the test before exploring both the websites and make assumptions wherever possible!
</div>

<div class="alert alert-block alert-info">
<b>Bonus:</b> Explore this notebook for bonus points and more hints
</div>

***
***
***

### Implementation

In [2]:
# Your code starts here

# Test 1

## Note : reCAPTCHA occurred and handeld manualy
### The next steps :
#### 1. Scrape the characteristics
#### 2. Store the data on DataFrame to clean and analyze it
#### 3. Store the data on MongoDB

In [144]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,\
    StaleElementReferenceException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
ignored_exceptions=(StaleElementReferenceException, NoSuchElementException, StaleElementReferenceException,\
                    TimeoutException, ElementNotInteractableException, ElementClickInterceptedException)


PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.implicitly_wait(10)


<ipython-input-144-48c7a3fce2ab>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [119]:
# create list of dictionaries for the first two privinces contain the first 150 streets 
province = [
    {"name": "Ajalvir", "link": "https://www.idealista.com/en/maps/ajalvir-madrid/", "streets": []},
    {"name": "Alameda del Valle", "link": "https://www.idealista.com/en/maps/alameda-del-valle-madrid/", "streets": []}
]


for p in province:
    time.sleep(3)
    driver.get(p['link'])
    # Streets Names
    streets_names1 = driver.find_elements(By.XPATH,'//li[@class="mb-4"]//span[1]')
    streets_names2 = driver.find_elements(By.XPATH,'//li[@class="mb-4"]//span[2]')
    streets_names = []
    for i in range(len(streets_names1)):
        streets_names.append(streets_names1[i].text+" "+streets_names2[i].text)
    # links to navigate to streets to scrape the buildings and store them
    links = driver.find_elements(By.XPATH,'//li[@class="mb-4"]//a')
    links_list = []
    for link in links: 
        links_list.append(link.get_attribute('href'))
    for i in range(len(streets_names)):
        p['streets'].append({
            "name": streets_names[i],
            "link": links_list[i],
            "buildings": []
    })

In [120]:
for p in province:
    time.sleep(3)
    for street in p['streets']:
        driver.get(street['link'])
        time.sleep(2)
        links = driver.find_elements(By.XPATH,'//*[@id="scroll__behavior"]/div[2]/div[2]/div[1]/a')    
        links_list = []
        # Get the links that displayed on Picture 2
        for link in links:
            street['buildings'].append({
            "link": link.get_attribute('href'),
            "Data": []
            })

In [175]:
for i in province:
    for s in i['streets']:
        for b in s['buildings']:
            time.sleep(3)
            driver.get(b['link'])
            b_data = driver.find_elements(By.XPATH,'//*[@id="scroll__behavior"]/div[2]/div[4]/div[1]/ol/li')
            try :
                b_id = driver.find_element(By.XPATH,'//*[@id="scroll__behavior"]/div[2]/div[3]/div/div/section/ol/li[1]/p[2]')
            except ignored_exceptions:
                try :
                    b_id = driver.find_element(By.XPATH,'//*[@id="scroll__behavior"]/div[2]/div[2]/div/div/section/ol/li[1]/p[2]')
                except ignored_exceptions:
                    continue
            res = {}
            res["id"] = b_id.text
            for d in range(len(b_data)):
                res["Data_"+str(d)] = b_data[d].text
            b['Data'].append(res)

In [177]:
final_list = []
for i in province:
    for s in i['streets']:
        for b in s['buildings']:
            final_list+=b["Data"]

In [179]:
df = pd.DataFrame(final_list)
df

,id,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8
0,9876903VK5897N0001QK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9876501VK5897N0014MT,Building with 3 floors,Built in 1997,"2,567 m² de parcela",Garage,Garden,Normal construction quality,1 Home,"Energy efficiency rating:\n(<303,7 kWh/m²/año)",NaN
2,9876902VK5897N0001GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9876501VK5897N0013XR,Building with 3 floors,Built in 1997,"2,567 m² de parcela",Garage,Garden,Normal construction quality,1 Home,"Energy efficiency rating:\n(<303,7 kWh/m²/año)",NaN
4,9876904VK5897N0001PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
646,9236741VK5893N0001BL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,9236742VK5893N0001YL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,9236743VK5893N0001GL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
649,9236744VK5893N0001QL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
df['id'] = df['id'].str[0:15]
df

,id,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8
0,9876903VK5897N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9876501VK5897N0,Building with 3 floors,Built in 1997,"2,567 m² de parcela",Garage,Garden,Normal construction quality,1 Home,"Energy efficiency rating:\n(<303,7 kWh/m²/año)",NaN
2,9876902VK5897N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9876501VK5897N0,Building with 3 floors,Built in 1997,"2,567 m² de parcela",Garage,Garden,Normal construction quality,1 Home,"Energy efficiency rating:\n(<303,7 kWh/m²/año)",NaN
4,9876904VK5897N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
646,9236741VK5893N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,9236742VK5893N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,9236743VK5893N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
649,9236744VK5893N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
import pandas as pd
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017")
Buildings_data = df.to_dict(orient = "records")
db = client["Madrid_Buildings"]
db.Iris.insert_many(Buildings_data)

<img src="../Tests/assets/Capture5.PNG" alt="Home" />
<h3 align="center">Picture-4</h3>

# Second Test

### Scrape all metro lines and thier stattions

In [276]:
# append list of dictionares for the metros
driver.get("https://www.metromadrid.es/en")

stations = []

lines_list = []
count=1

links = driver.find_elements(By.XPATH,'//li[@class="list__lineas__element"]//a')

for link in links:
    time.sleep(3)
    stations.append({
        "line": count,
        "link": link.get_attribute('href'),
        "stations": []
    })
    count+=1

for station in stations:
    time.sleep(3)
    driver.get(station["link"])
    stations_names = driver.find_elements(By.XPATH,'//a[@class="list-line__btn accordion-title"]/p')
    for station_name in stations_names:
        if station_name.text + " Station" in station["stations"]:
            pass
        else:
            station["stations"].append(station_name.text + " Station")

<ipython-input-276-e9c4ab7d32e4>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


## Q1 : Total number of the stations is 276

In [306]:
df = pd.DataFrame(stations)
stations_list = []
for i in range(len(df)):
    station = df["stations"][i]
    for y in range(len(station)):
        station_ = station[y]
        if station_ in stations_list:
            pass
        else :
            stations_list.append(station_)
print(len(stations_list))

276


### Travel time to Pinar de Chamartín

In [280]:
driver.get("https://ctmulti.metromadrid.es/ctmulti/accesytevisor/index.html")
time.sleep(60)
# The starting Station
start = "Pinar de Chamartín Station"
travel_times_to_Pinar = []
time.sleep(5)
start_station = driver.find_element(By.XPATH,'(//input[@class="searchInput"])[2]')
time.sleep(5)
start_station.send_keys(start)
time.sleep(5)
start_station.send_keys(Keys.ENTER)
time.sleep(5)
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException,TimeoutException)
end_station = WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
                    .until(EC.presence_of_element_located((By.XPATH, '(//input[@class="searchInput"])[3]')))
# Looping inside the stations list to difine the destination
for station in stations_list:
    time.sleep(3)
    if start == station:
        pass
    else :
        end_station.send_keys(station)
        sug = WebDriverWait(driver, 50,ignored_exceptions=ignored_exceptions)\
                    .until(EC.element_to_be_clickable((By.XPATH, '(//li[@data-index="0"]//strong)[2][1][1]')))
        sug.click()
        travel_time = WebDriverWait(driver, 50,ignored_exceptions=ignored_exceptions)\
                    .until(EC.presence_of_element_located((By.XPATH, '//div[@class="esriImpedanceCost"]')))
        time.sleep(3)
        travel_times_to_Pinar.append({
            "Start" : "Pinar de Chamartín",
            "End": station,
            "Travel Time": travel_time.text
        })
        end_station.clear()

<ipython-input-280-0c72caa24a0a>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [353]:
# Converting the list of dicts to DataFrame to analyize the data
Time_to_Pinar_DF = pd.DataFrame(travel_times_to_Pinar)
# Exstracting the Travel time in numerical values
for i in range(len(Time_to_Pinar_DF)):
    Time_to_Pinar_DF['Travel Time'][i] = (int(Time_to_Pinar_DF['Travel Time'][i][0])*60)+(int(Time_to_Pinar_DF['Travel Time'][i][4:6]))

## nearest and farthest stations to Pinar de Chamartín
### Nearest is Chamartín Station : with 3 min travel time
### Farthest is Puerta de Boadilla Station : with 81 mins travel time

In [358]:
print(Time_to_Pinar_DF[Time_to_Pinar_DF["Travel Time"] == Time_to_Pinar_DF["Travel Time"].max()])
print(Time_to_Pinar_DF[Time_to_Pinar_DF["Travel Time"] == Time_to_Pinar_DF["Travel Time"].max()])

                  Start                         End Travel Time  \
274  Pinar de Chamartín  Puerta de Boadilla Station          81   

                          _id  
274  61c5e905b9d2e4dc1232d0d4  
                  Start                         End Travel Time  \
274  Pinar de Chamartín  Puerta de Boadilla Station          81   

                          _id  
274  61c5e905b9d2e4dc1232d0d4  


### Save the data in mongodb

In [357]:
import pandas as pd
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017")
data = Time_to_Pinar_DF.to_dict(orient = "records")
db = client["Pinar_de_Chamartín"]
db.Iris.insert_many(data)

<img src="../Tests/assets/Capture.PNG" alt="Home" />
<h3 align="center">Picture-4</h3>

### Travel Time to Las Rosas

In [310]:
driver.get("https://ctmulti.metromadrid.es/ctmulti/accesytevisor/index.html")
time.sleep(60)
# The starting Station
start = "Las Rosas Station"
travel_times_to_Las_Rosas = []
time.sleep(5)
start_station = driver.find_element(By.XPATH,'(//input[@class="searchInput"])[2]')
time.sleep(5)
start_station.send_keys(start)
time.sleep(5)
start_station.send_keys(Keys.ENTER)
time.sleep(5)
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException,TimeoutException)
end_station = WebDriverWait(driver, 15,ignored_exceptions=ignored_exceptions)\
                    .until(EC.presence_of_element_located((By.XPATH, '(//input[@class="searchInput"])[3]')))
# Looping inside the stations list to difine the destination
for station in stations_list:
    time.sleep(3)
    if start == station:
        pass
    else :
        end_station.send_keys(station)
        sug = WebDriverWait(driver, 50,ignored_exceptions=ignored_exceptions)\
                    .until(EC.element_to_be_clickable((By.XPATH, '(//li[@data-index="0"]//strong)[2][1][1]')))
        sug.click()
        travel_time = WebDriverWait(driver, 50,ignored_exceptions=ignored_exceptions)\
                    .until(EC.presence_of_element_located((By.XPATH, '//div[@class="esriImpedanceCost"]')))
        time.sleep(3)
        travel_times_to_Las_Rosas.append({
            "Start" : "Las Rosas",
            "End": station,
            "Travel Time": travel_time.text
        })
        end_station.clear()

<ipython-input-310-48aa9fe77ae2>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [333]:
# Converting the list of dicts to DataFrame to analyize the data
Time_to_Las_Rosas_DF = pd.DataFrame(travel_times_to_Las_Rosas)
Time_to_Las_Rosas_DF
# Exstracting the Travel time in numerical values
for i in range(len(Time_to_Las_Rosas_DF)):
    Time_to_Las_Rosas_DF['Travel Time'][i] = str(Time_to_Las_Rosas_DF['Travel Time'][i])
    try:
        Time_to_Las_Rosas_DF['Travel Time'][i] = (int(Time_to_Las_Rosas_DF['Travel Time'][i][0])*60)+(int(Time_to_Las_Rosas_DF['Travel Time'][i][4:6]))
    except (ValueError,TypeError ):
        Time_to_Las_Rosas_DF['Travel Time'][i] = (int(Time_to_Las_Rosas_DF['Travel Time'][i][0])*60)
Time_to_Las_Rosas_DF

,Start,End,Travel Time
0,Las Rosas,Pinar de Chamartín Station,44
1,Las Rosas,Bambú Station,41
2,Las Rosas,Chamartín Station,44
3,Las Rosas,Plaza de Castilla Station,36
4,Las Rosas,Valdeacederas Station,38
...,...,...,...
270,Las Rosas,Boadilla Centro Station,82
271,Las Rosas,Nuevo Mundo Station,84
272,Las Rosas,Siglo XXI Station,86
273,Las Rosas,Infante Don Luís Station,88


### Average time from Las Rosas to all stations is 43.6 mins

In [337]:
AVG = Time_to_Las_Rosas_DF['Travel Time'].mean()
AVG

43.596363636363634

In [338]:
import pandas as pd
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017")
data = Time_to_Las_Rosas_DF.to_dict(orient = "records")
db = client["Las_Rosas"]
db.Iris.insert_many(data)

<img src="../Tests/assets/Capture1.PNG" alt="Home" />
<h3 align="center">Picture-4</h3>